In [2]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives.nq", format='nquads')

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""
 
sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(my_SPARQL_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])

http://wikiba.se/ontology#Dump http://creativecommons.org/ns#license http://creativecommons.org/publicdomain/zero/1.0/
http://wikiba.se/ontology#Dump http://schema.org/softwareVersion 1.0.0
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:00:02Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:15Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:16Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:19Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:20Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:24Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:40Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:41Z


In [4]:
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/")
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")


arthistorians_list = set()


for s,p,o in g.triples(( None, wdt.P170, None)):  
    if "wikidata.org/entity/" in str(o):           
        arthistorians_list.add('<' + str(o) + '>')     
    
print(arthistorians_list)

{'<http://www.wikidata.org/entity/Q18935222>', '<http://www.wikidata.org/entity/Q55453618>', '<http://www.wikidata.org/entity/Q90407>', '<http://www.wikidata.org/entity/Q1271052>', '<http://www.wikidata.org/entity/Q19997512>', '<http://www.wikidata.org/entity/Q88907>', '<http://www.wikidata.org/entity/Q61913691>', '<http://www.wikidata.org/entity/Q457739>', '<http://www.wikidata.org/entity/Q3051533>', '<http://www.wikidata.org/entity/Q2824734>', '<http://www.wikidata.org/entity/Q6700132>', '<http://www.wikidata.org/entity/Q1373290>', '<http://www.wikidata.org/entity/Q1089074>', '<http://www.wikidata.org/entity/Q1641821>', '<http://www.wikidata.org/entity/Q85761254>', '<http://www.wikidata.org/entity/Q1629748>', '<http://www.wikidata.org/entity/Q41616785>', '<http://www.wikidata.org/entity/Q995470>', '<http://www.wikidata.org/entity/Q60185>', '<http://www.wikidata.org/entity/Q1296486>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q1715096>', '<http://www

In [5]:
historians = ' '.join(arthistorians_list) 


university_query = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT DISTINCT ?historian ?university ?university_label
WHERE {
    VALUES ?historian {"""+historians+"""} . # look how we include a variable in a query string!
    ?historian wdt:P69 ?university . 
    ?university rdfs:label ?university_label .
    FILTER (langMatches(lang(?university_label), "EN"))
    } 
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(university_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    historian_uri = result["historian"]["value"]
    print("historian:", historian_uri)
    if "university" in result: 
        university = result["university"]["value"]
        if "university_label" in result: 
            university_label = result["university_label"]["value"]
            print("found:", university, university_label)
            
            
            g.add(( URIRef(historian_uri) , URIRef(wdt.P69) , URIRef(university) ))
            g.add(( URIRef(university) , RDFS.label , Literal(university_label) ))
    else:
        print("nothing found in wikidata :(")

historian: http://www.wikidata.org/entity/Q19997512
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q19997512
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q19997512
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q3051533
found: http://www.wikidata.org/entity/Q21578 Princeton University
historian: http://www.wikidata.org/entity/Q3051533
found: http://www.wikidata.org/entity/Q21578 Princeton University
historian: http://www.wikidata.org/entity/Q3051533
found: http://www.wikidata.org/entity/Q21578 Princeton University
historian: http://www.wikidata.org/entity/Q88907
found: http://www.wikidata.org/entity/Q55044 Ludwig Maximilian University of Munich
historian: http://www.wikidata.org/entity/Q88907
found: http://www.wikidata.org/entity/Q55044 Ludwig Maximilian University of Munich
historian: http://www.wikidata.org/

In [6]:
g.serialize(destination='artchives_P69.nq', format='nquads')

In [7]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/")
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives_P69.nq", format='nquads')

In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""

 
sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(my_SPARQL_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])

http://wikiba.se/ontology#Dump http://creativecommons.org/ns#license http://creativecommons.org/publicdomain/zero/1.0/
http://wikiba.se/ontology#Dump http://schema.org/softwareVersion 1.0.0
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:00:02Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:15Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:16Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:19Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:20Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:24Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:40Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:41Z


In [9]:
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")


arthistorians_list = set()


for s,p,o in g.triples(( None, wdt.P170, None)):  
    if "wikidata.org/entity/" in str(o):           
        arthistorians_list.add('<' + str(o) + '>')    
    
print(arthistorians_list)

{'<http://www.wikidata.org/entity/Q18935222>', '<http://www.wikidata.org/entity/Q55453618>', '<http://www.wikidata.org/entity/Q90407>', '<http://www.wikidata.org/entity/Q1271052>', '<http://www.wikidata.org/entity/Q19997512>', '<http://www.wikidata.org/entity/Q88907>', '<http://www.wikidata.org/entity/Q61913691>', '<http://www.wikidata.org/entity/Q457739>', '<http://www.wikidata.org/entity/Q3051533>', '<http://www.wikidata.org/entity/Q2824734>', '<http://www.wikidata.org/entity/Q6700132>', '<http://www.wikidata.org/entity/Q1373290>', '<http://www.wikidata.org/entity/Q1089074>', '<http://www.wikidata.org/entity/Q1641821>', '<http://www.wikidata.org/entity/Q85761254>', '<http://www.wikidata.org/entity/Q1629748>', '<http://www.wikidata.org/entity/Q41616785>', '<http://www.wikidata.org/entity/Q995470>', '<http://www.wikidata.org/entity/Q60185>', '<http://www.wikidata.org/entity/Q1296486>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q1715096>', '<http://www

In [10]:
historians = ' '.join(arthistorians_list) 

employer_query = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT DISTINCT ?historian ?employer ?employer_label
WHERE {
    VALUES ?historian {"""+historians+"""} . # look how we include a variable in a query string!
    ?historian wdt:P108 ?employer . 
    ?employer rdfs:label ?employer_label .
    FILTER (langMatches(lang(?employer_label), "EN"))
    } 
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(employer_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    historian_uri = result["historian"]["value"]
    print("historian:", historian_uri)
    if "employer" in result: 
        employer = result["employer"]["value"]
        if "employer_label" in result: 
            employer_label = result["employer_label"]["value"]
            print("found:", employer, employer_label)
            
           
            g.add(( URIRef(historian_uri) , URIRef(wdt.P108) , URIRef(employer) ))
            g.add(( URIRef(employer) , RDFS.label , Literal(employer_label) ))
    else:
        print("nothing found in wikidata :(")

historian: http://www.wikidata.org/entity/Q88907
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q88907
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q88907
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q457739
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q457739
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q457739
found: http://www.wikidata.org/entity/Q13371 Harvard University
historian: http://www.wikidata.org/entity/Q537874
found: http://www.wikidata.org/entity/Q34433 University of Oxford
historian: http://www.wikidata.org/entity/Q537874
found: http://www.wikidata.org/entity/Q34433 University of Oxford
historian: http://www.wikidata.org/entity/Q537874
found: http://www.wikidata.org/entity

In [11]:
g.serialize(destination='artchives_P108.nq', format='nquads')

In [12]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives_P108.nq", format='nquads')

In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""

sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(my_SPARQL_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])

http://wikiba.se/ontology#Dump http://creativecommons.org/ns#license http://creativecommons.org/publicdomain/zero/1.0/
http://wikiba.se/ontology#Dump http://schema.org/softwareVersion 1.0.0
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:00:02Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:15Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:16Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:19Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:20Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:24Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:40Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:41Z


In [14]:
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS

wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
rdfs = Namespace ("http://www.w3.org/2000/01/rdf-schema#")
rdf = Namespace ("http://www.w3.org/1999/02/22-rdf-syntax-ns#")


university_list = set()


for s,p,o in g.triples(( None, wdt.P108, None)):   
    if "wikidata.org/entity/" in str(o):           
        university_list.add('<' + str(o) + '>')     
for s,p,o in g.triples(( None, wdt.P69, None)):   
    if "wikidata.org/entity/" in str(o):           
        university_list.add('<' + str(o) + '>')     
    
print(university_list)

{'<http://www.wikidata.org/entity/Q49088>', '<http://www.wikidata.org/entity/Q5303281>', '<http://www.wikidata.org/entity/Q153987>', '<http://www.wikidata.org/entity/Q1399299>', '<http://www.wikidata.org/entity/Q32120>', '<http://www.wikidata.org/entity/Q55044>', '<http://www.wikidata.org/entity/Q223429>', '<http://www.wikidata.org/entity/Q152087>', '<http://www.wikidata.org/entity/Q154804>', '<http://www.wikidata.org/entity/Q1446181>', '<http://www.wikidata.org/entity/Q21578>', '<http://www.wikidata.org/entity/Q213439>', '<http://www.wikidata.org/entity/Q3558578>', '<http://www.wikidata.org/entity/Q155354>', '<http://www.wikidata.org/entity/Q35794>', '<http://www.wikidata.org/entity/Q34433>', '<http://www.wikidata.org/entity/Q152171>', '<http://www.wikidata.org/entity/Q1376987>', '<http://www.wikidata.org/entity/Q230899>', '<http://www.wikidata.org/entity/Q1360303>', '<http://www.wikidata.org/entity/Q503176>', '<http://www.wikidata.org/entity/Q3339070>', '<http://www.wikidata.org/enti

In [15]:
university = ' '.join(university_list) 

university_triples_query = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX art: <https://w3id.org/artchives/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?university ?university_label
WHERE {
    VALUES ?university {"""+university+"""} . # look how we include a variable in a query string!
   
    ?university rdfs:label ?university_label .
    FILTER (langMatches(lang(?university_label), "EN"))
    } 
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(university_triples_query )

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    university_uri = result["university"]["value"]
    print("university:", university_uri)
    if "university" in result: 
        university = result["university"]["value"]
        if "university_label" in result: 
            university_label = result["university_label"]["value"]
            print("found:", university, university_label)
            
            
            g.add(( URIRef(university_uri) , URIRef(rdf.type) , URIRef(wd.Q3918) ))
            g.add(( URIRef(university) , RDFS.label , Literal(university_label) ))
    else:
        print("nothing found in wikidata :(")

university: http://www.wikidata.org/entity/Q13371
found: http://www.wikidata.org/entity/Q13371 Harvard University
university: http://www.wikidata.org/entity/Q13371
found: http://www.wikidata.org/entity/Q13371 Harvard University
university: http://www.wikidata.org/entity/Q13371
found: http://www.wikidata.org/entity/Q13371 Harvard University
university: http://www.wikidata.org/entity/Q21578
found: http://www.wikidata.org/entity/Q21578 Princeton University
university: http://www.wikidata.org/entity/Q21578
found: http://www.wikidata.org/entity/Q21578 Princeton University
university: http://www.wikidata.org/entity/Q21578
found: http://www.wikidata.org/entity/Q21578 Princeton University
university: http://www.wikidata.org/entity/Q32120
found: http://www.wikidata.org/entity/Q32120 University of Halle-Wittenberg
university: http://www.wikidata.org/entity/Q34433
found: http://www.wikidata.org/entity/Q34433 University of Oxford
university: http://www.wikidata.org/entity/Q34433
found: http://www.

In [16]:
g.serialize(destination='artchives_unitype_unilabel.nq', format='nquads')

In [17]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives_unitype_unilabel.nq", format='nquads')

In [18]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""

 
sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(my_SPARQL_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])

http://wikiba.se/ontology#Dump http://creativecommons.org/ns#license http://creativecommons.org/publicdomain/zero/1.0/
http://wikiba.se/ontology#Dump http://schema.org/softwareVersion 1.0.0
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:00:02Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:15Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:16Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:19Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:20Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:24Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:40Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:41Z


In [19]:
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
rdfs = Namespace ("http://www.w3.org/2000/01/rdf-schema#")
rdf = Namespace ("http://www.w3.org/1999/02/22-rdf-syntax-ns#")


university_list = set()


for s,p,o in g.triples(( None, wdt.P108, None)):   
    if "wikidata.org/entity/" in str(o):           
        university_list.add('<' + str(o) + '>')     
for s,p,o in g.triples(( None, wdt.P69, None)):   
    if "wikidata.org/entity/" in str(o):           
        university_list.add('<' + str(o) + '>')      
    
print(university_list)

{'<http://www.wikidata.org/entity/Q49088>', '<http://www.wikidata.org/entity/Q5303281>', '<http://www.wikidata.org/entity/Q153987>', '<http://www.wikidata.org/entity/Q32120>', '<http://www.wikidata.org/entity/Q1399299>', '<http://www.wikidata.org/entity/Q55044>', '<http://www.wikidata.org/entity/Q223429>', '<http://www.wikidata.org/entity/Q152087>', '<http://www.wikidata.org/entity/Q154804>', '<http://www.wikidata.org/entity/Q1446181>', '<http://www.wikidata.org/entity/Q213439>', '<http://www.wikidata.org/entity/Q21578>', '<http://www.wikidata.org/entity/Q3558578>', '<http://www.wikidata.org/entity/Q155354>', '<http://www.wikidata.org/entity/Q35794>', '<http://www.wikidata.org/entity/Q34433>', '<http://www.wikidata.org/entity/Q152171>', '<http://www.wikidata.org/entity/Q1376987>', '<http://www.wikidata.org/entity/Q230899>', '<http://www.wikidata.org/entity/Q1360303>', '<http://www.wikidata.org/entity/Q503176>', '<http://www.wikidata.org/entity/Q3339070>', '<http://www.wikidata.org/enti

In [20]:
universities = ' '.join(university_list) 


university_country_query = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
SELECT DISTINCT ?university ?country ?country_label
WHERE {
    VALUES ?university {"""+universities+"""} . # look how we include a variable in a query string!
    ?university wdt:P17 ?country . 
    ?country rdfs:label ?country_label .
    FILTER (langMatches(lang(?country_label), "EN"))
    } 
"""

 
sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(university_country_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    university_uri = result["university"]["value"]
    print("university:", university_uri)
    if "country" in result:
        country = result["country"]["value"]
        if "country_label" in result: 
            country_label = result["country_label"]["value"]
            print("found:", country, country_label)
            
            
            g.add(( URIRef(university_uri) , URIRef(wdt.P17) , URIRef(country) ))
            g.add(( URIRef(country) , RDFS.label , Literal(country_label) ))
    else:
        print("nothing found in wikidata :(")

university: http://www.wikidata.org/entity/Q131252
found: http://www.wikidata.org/entity/Q30 United States of America
university: http://www.wikidata.org/entity/Q131252
found: http://www.wikidata.org/entity/Q30 United States
university: http://www.wikidata.org/entity/Q131252
found: http://www.wikidata.org/entity/Q30 United States of America
university: http://www.wikidata.org/entity/Q167733
found: http://www.wikidata.org/entity/Q30 United States of America
university: http://www.wikidata.org/entity/Q167733
found: http://www.wikidata.org/entity/Q30 United States
university: http://www.wikidata.org/entity/Q167733
found: http://www.wikidata.org/entity/Q30 United States of America
university: http://www.wikidata.org/entity/Q213439
found: http://www.wikidata.org/entity/Q30 United States of America
university: http://www.wikidata.org/entity/Q213439
found: http://www.wikidata.org/entity/Q30 United States
university: http://www.wikidata.org/entity/Q213439
found: http://www.wikidata.org/entity/

university: http://www.wikidata.org/entity/Q156725
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q156725
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q156725
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q32120
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q32120
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q32120
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q55044
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q55044
found: http://www.wikidata.org/entity/Q183 Germany
university: http://www.wikidata.org/entity/Q55044
found: http://www.wikidata.org/entity/Q183 Germany


In [21]:
g.serialize(destination='artchives_P17.nq', format='nquads')

In [22]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives_P17.nq", format='nquads')

In [23]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(my_SPARQL_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])

http://wikiba.se/ontology#Dump http://creativecommons.org/ns#license http://creativecommons.org/publicdomain/zero/1.0/
http://wikiba.se/ontology#Dump http://schema.org/softwareVersion 1.0.0
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:00:02Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:15Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:16Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:19Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:20Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:24Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:40Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:41Z


In [24]:
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
rdfs = Namespace ("http://www.w3.org/2000/01/rdf-schema#")
rdf = Namespace ("http://www.w3.org/1999/02/22-rdf-syntax-ns#")


university_list = set()

# iterate over the triples in the graph
for s,p,o in g.triples(( None, wdt.P108, None)):  
    if "wikidata.org/entity/" in str(o):           
        university_list.add('<' + str(o) + '>')    
for s,p,o in g.triples(( None, wdt.P69, None)):   
    if "wikidata.org/entity/" in str(o):           
        university_list.add('<' + str(o) + '>')    
    
print(university_list)

{'<http://www.wikidata.org/entity/Q49088>', '<http://www.wikidata.org/entity/Q5303281>', '<http://www.wikidata.org/entity/Q153987>', '<http://www.wikidata.org/entity/Q32120>', '<http://www.wikidata.org/entity/Q1399299>', '<http://www.wikidata.org/entity/Q55044>', '<http://www.wikidata.org/entity/Q223429>', '<http://www.wikidata.org/entity/Q152087>', '<http://www.wikidata.org/entity/Q154804>', '<http://www.wikidata.org/entity/Q1446181>', '<http://www.wikidata.org/entity/Q213439>', '<http://www.wikidata.org/entity/Q21578>', '<http://www.wikidata.org/entity/Q3558578>', '<http://www.wikidata.org/entity/Q155354>', '<http://www.wikidata.org/entity/Q35794>', '<http://www.wikidata.org/entity/Q34433>', '<http://www.wikidata.org/entity/Q152171>', '<http://www.wikidata.org/entity/Q1376987>', '<http://www.wikidata.org/entity/Q230899>', '<http://www.wikidata.org/entity/Q1360303>', '<http://www.wikidata.org/entity/Q503176>', '<http://www.wikidata.org/entity/Q3339070>', '<http://www.wikidata.org/enti

In [25]:
university = ' '.join(university_list)


university_coordinate_query = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
SELECT DISTINCT ?university ?coordinate ?coordinate_label
WHERE {
    VALUES ?university {"""+university+"""} . # look how we include a variable in a query string!
    ?university wdt:P159 ?coordinate . 
    ?coordinate rdfs:label ?coordinate_label .
    FILTER (langMatches(lang(?coordinate_label), "EN"))
    } 
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(university_coordinate_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    university_uri = result["university"]["value"]
    print("university:", university_uri)
    if "coordinate" in result:
        coordinate = result["coordinate"]["value"]
        if "coordinate_label" in result: 
            coordinate_label = result["coordinate_label"]["value"]
            print("found:", coordinate, coordinate_label)
            
           
            g.add(( URIRef(university_uri) , URIRef(wdt.P159) , URIRef(coordinate) ))
            g.add(( URIRef(coordinate) , RDFS.label , Literal(coordinate_label) ))
    else:
        print("nothing found in wikidata :(")

university: http://www.wikidata.org/entity/Q503176
found: http://www.wikidata.org/entity/Q60 New York City
university: http://www.wikidata.org/entity/Q503176
found: http://www.wikidata.org/entity/Q60 New York City
university: http://www.wikidata.org/entity/Q503176
found: http://www.wikidata.org/entity/Q60 New York City
university: http://www.wikidata.org/entity/Q503176
found: http://www.wikidata.org/entity/Q84 London
university: http://www.wikidata.org/entity/Q503176
found: http://www.wikidata.org/entity/Q84 London
university: http://www.wikidata.org/entity/Q503176
found: http://www.wikidata.org/entity/Q84 London
university: http://www.wikidata.org/entity/Q1031803
found: http://www.wikidata.org/entity/Q220 Rome
university: http://www.wikidata.org/entity/Q35794
found: http://www.wikidata.org/entity/Q350 Cambridge
university: http://www.wikidata.org/entity/Q35794
found: http://www.wikidata.org/entity/Q350 Cambridge
university: http://www.wikidata.org/entity/Q35794
found: http://www.wikid

In [26]:
g.serialize(destination='artchives_P159.nq', format='nquads')

In [27]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives_P159.nq", format='nquads')

In [28]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"

my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(my_SPARQL_query)

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])

http://wikiba.se/ontology#Dump http://creativecommons.org/ns#license http://creativecommons.org/publicdomain/zero/1.0/
http://wikiba.se/ontology#Dump http://schema.org/softwareVersion 1.0.0
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:00:02Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:15Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:16Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:19Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:20Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:24:24Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:40Z
http://wikiba.se/ontology#Dump http://schema.org/dateModified 2020-03-02T23:51:41Z


In [29]:

from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS


wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
rdfs = Namespace ("http://www.w3.org/2000/01/rdf-schema#")
rdf = Namespace ("http://www.w3.org/1999/02/22-rdf-syntax-ns#")


citty_list = set()


for s,p,o in g.triples(( None, wdt.P159, None)):   
    if "wikidata.org/entity/" in str(o):          
        citty_list.add('<' + str(o) + '>')     
   
    
print(citty_list)

{'<http://www.wikidata.org/entity/Q18125>', '<http://www.wikidata.org/entity/Q2047280>', '<http://www.wikidata.org/entity/Q34217>', '<http://www.wikidata.org/entity/Q84>', '<http://www.wikidata.org/entity/Q49111>', '<http://www.wikidata.org/entity/Q1055>', '<http://www.wikidata.org/entity/Q60>', '<http://www.wikidata.org/entity/Q2256>', '<http://www.wikidata.org/entity/Q138518>', '<http://www.wikidata.org/entity/Q2079>', '<http://www.wikidata.org/entity/Q123766>', '<http://www.wikidata.org/entity/Q3869>', '<http://www.wikidata.org/entity/Q220>', '<http://www.wikidata.org/entity/Q49241>', '<http://www.wikidata.org/entity/Q1449>', '<http://www.wikidata.org/entity/Q350>', '<http://www.wikidata.org/entity/Q11299>', '<http://www.wikidata.org/entity/Q102114571>', '<http://www.wikidata.org/entity/Q1589473>', '<http://www.wikidata.org/entity/Q586>', '<http://www.wikidata.org/entity/Q2044>'}


In [30]:
citty = ' '.join(citty_list) 


citty_triples_query = """
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX art: <https://w3id.org/artchives/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?citty ?citty_label
WHERE {
    VALUES ?citty {"""+citty+"""} . # look how we include a variable in a query string!
   
    ?citty rdfs:label ?citty_label .
    FILTER (langMatches(lang(?citty_label), "EN"))
    } 
"""


sparql_wd = SPARQLWrapper(wikidata_endpoint)

sparql_wd.setQuery(citty_triples_query )

sparql_wd.setReturnFormat(JSON)

results = sparql_wd.query().convert()


for result in results["results"]["bindings"]:
    citty_uri = result["citty"]["value"]
    print("citty:", citty_uri)
    if "citty" in result: 
        citty = result["citty"]["value"]
        if "citty_label" in result: 
            citty_label = result["citty_label"]["value"]
            print("found:", citty, citty_label)
            
            
            g.add(( URIRef(citty_uri) , URIRef(rdf.type) , URIRef(wd.Q515) ))
            g.add(( URIRef(citty) , RDFS.label , Literal(citty_label) ))
    else:
        print("nothing found in wikidata :(")

citty: http://www.wikidata.org/entity/Q60
found: http://www.wikidata.org/entity/Q60 New York City
citty: http://www.wikidata.org/entity/Q60
found: http://www.wikidata.org/entity/Q60 New York City
citty: http://www.wikidata.org/entity/Q60
found: http://www.wikidata.org/entity/Q60 New York City
citty: http://www.wikidata.org/entity/Q84
found: http://www.wikidata.org/entity/Q84 London
citty: http://www.wikidata.org/entity/Q84
found: http://www.wikidata.org/entity/Q84 London
citty: http://www.wikidata.org/entity/Q84
found: http://www.wikidata.org/entity/Q84 London
citty: http://www.wikidata.org/entity/Q220
found: http://www.wikidata.org/entity/Q220 Rome
citty: http://www.wikidata.org/entity/Q586
found: http://www.wikidata.org/entity/Q586 Bonn
citty: http://www.wikidata.org/entity/Q586
found: http://www.wikidata.org/entity/Q586 Bonn
citty: http://www.wikidata.org/entity/Q586
found: http://www.wikidata.org/entity/Q586 Bonn
citty: http://www.wikidata.org/entity/Q1055
found: http://www.wikidat

In [31]:
g.serialize(destination='artchives_citytype.nq', format='nquads')

In [32]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

In [33]:
wd = Namespace("http://www.wikidata.org/entity/") 
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
xs = Namespace("http://www.w3.org/2001/XMLSchema#")


g = rdflib.ConjunctiveGraph()


result = g.parse("artchives_citytype.nq", format='nquads')



predicate = article = rdflib.term.URIRef("http://www.w3.org/2000/01/rdf-schema#label")



triples_set= set()

for s,p,o in g.triples((None,predicate,None)):
    if s in triples_set:
        g.remove((s,p,o))
        print("i've removed this triple:", s,p,o)
    else:
        print("i DIDN'T remove this one:", s,p,o)
        triples_set.add(s)
        
print("These should be unique triples:", g.triples((None,predicate,None)))
g.serialize(destination='artchives_output.nq', format='nquads')
        


i DIDN'T remove this one: http://www.wikidata.org/entity/Q1060159 http://www.w3.org/2000/01/rdf-schema#label Thomas Wayne
i DIDN'T remove this one: https://w3id.org/artchives/biblioOnArtHistorian http://www.w3.org/2000/01/rdf-schema#label Selected Bibliography:
Agamben, G. (1984). Aby Warburg e la scienza senza nome. In "aut aut", (199-200, gennaio-aprile 1984, pp. 51-66);
Cieri Via, C. (2011). Introduzione a Aby Warburg. Bari: Laterza;
Cieri Via, C. (Ed.) (2009). Aby Warburg e la cultura italiana: 	fra sopravvivenze e prospettive di ricerca. Milano: Mondadori;
Aby Warburg. La dialettica dell'immagine. "aut aut" n. 321-322, maggio-agosto 2004;
Gombrich, E. (1970). Aby Warburg. An intellectual Biography. London: Phaidon;
Ginzburg, C. (1986). Da A. Warburg a E. H. Gombrich. Note su un problema di metodo. In: Ginzburg, C. (1986). Miti emblemi spie: Morfologia e storia, Torino: Einaudi;
Didi-Huberman, G. (2006). L'immagine insepolta. Aby Warburg, la memoria dei fantasmi e la storia dell'ar

i DIDN'T remove this one: https://w3id.org/artchives/MD1559576252213 http://www.w3.org/2000/01/rdf-schema#label                      R.F. Brown
i DIDN'T remove this one: http://www.wikidata.org/entity/Q47163 http://www.w3.org/2000/01/rdf-schema#label Vitruvius
i DIDN'T remove this one: https://w3id.org/artchives/MD1559580909651 http://www.w3.org/2000/01/rdf-schema#label                 Casini Edizioni
i DIDN'T remove this one: http://www.wikidata.org/entity/Q107970 http://www.w3.org/2000/01/rdf-schema#label    Paul Frankl
i DIDN'T remove this one: http://www.wikidata.org/entity/Q22669857 http://www.w3.org/2000/01/rdf-schema#label                  collage
i DIDN'T remove this one: https://w3id.org/artchives/MD1559579729830 http://www.w3.org/2000/01/rdf-schema#label                   B. Cole
i DIDN'T remove this one: http://www.wikidata.org/entity/Q34433 http://www.w3.org/2000/01/rdf-schema#label University of Oxford
i've removed this triple: http://www.wikidata.org/entity/Q125191 http:/

i DIDN'T remove this one: https://w3id.org/artchives/1582011722-53/ http://www.w3.org/2000/01/rdf-schema#label Wolfgang Lotz
i DIDN'T remove this one: https://w3id.org/artchives/MD1559563478513 http://www.w3.org/2000/01/rdf-schema#label                            B. Bearzi
i DIDN'T remove this one: http://www.wikidata.org/entity/Q5263780 http://www.w3.org/2000/01/rdf-schema#label            Describing Archives: A Content Standard
i DIDN'T remove this one: http://www.wikidata.org/entity/Q3907879 http://www.w3.org/2000/01/rdf-schema#label        Pompeo Molmenti
i've removed this triple: https://w3id.org/artchives/biblioOnArtHistorian http://www.w3.org/2000/01/rdf-schema#label Mackowsky, H. (1910), Cornel von Fabriczy, in Repertorium für Kunstwissenschaft, (Vol. 33.1910, pp. 485-492). Berlin: Georg Reimer;
Cornelio de Fabriczy (1910): [necrologio, a cura di alcuni amici fiorentini], con la bibliografia dei principali scritti di C. De Fabriczy sull'arte italiana. Sonderdruck aus : Rivista 

i DIDN'T remove this one: https://w3id.org/artchives/MD1559564552900 http://www.w3.org/2000/01/rdf-schema#label                          Th. Boal
i DIDN'T remove this one: https://w3id.org/artchives/MD1559579321700 http://www.w3.org/2000/01/rdf-schema#label                    B. Cherubini
i DIDN'T remove this one: https://w3id.org/artchives/MD1559659424927 http://www.w3.org/2000/01/rdf-schema#label            F. Faber-Spälty
i've removed this triple: http://www.wikidata.org/entity/Q3494271 http://www.w3.org/2000/01/rdf-schema#label                archive document
i DIDN'T remove this one: https://w3id.org/artchives/MD1559657971583 http://www.w3.org/2000/01/rdf-schema#label              D. Donald
i DIDN'T remove this one: https://w3id.org/artchives/MD1557412258614 http://www.w3.org/2000/01/rdf-schema#label                Certani collection
i DIDN'T remove this one: https://w3id.org/artchives/MD1559576033738 http://www.w3.org/2000/01/rdf-schema#label                      A. Brand
i've re

i've removed this triple: http://www.wikidata.org/entity/Q172984 http://www.w3.org/2000/01/rdf-schema#label          gelatin silver process
i DIDN'T remove this one: http://www.wikidata.org/entity/Q841090 http://www.w3.org/2000/01/rdf-schema#label classics
i DIDN'T remove this one: https://w3id.org/artchives/MD1559576179201 http://www.w3.org/2000/01/rdf-schema#label                      H. Brinkmann
i've removed this triple: http://www.wikidata.org/entity/Q1654544 http://www.w3.org/2000/01/rdf-schema#label         ISAD(G)
i DIDN'T remove this one: https://w3id.org/artchives/MD1559660216880 http://www.w3.org/2000/01/rdf-schema#label            L. Fröhlich-Bume
i DIDN'T remove this one: https://w3id.org/artchives/MD1559579908224 http://www.w3.org/2000/01/rdf-schema#label                   M.E. Cope
i DIDN'T remove this one: http://www.wikidata.org/entity/Q1210 http://www.w3.org/2000/01/rdf-schema#label Lombardy
i DIDN'T remove this one: http://www.wikidata.org/entity/Q4868805 http://www.

i've removed this triple: http://www.wikidata.org/entity/Q5599 http://www.w3.org/2000/01/rdf-schema#label Peter Paul Rubens
i DIDN'T remove this one: http://www.wikidata.org/entity/Q742333 http://www.w3.org/2000/01/rdf-schema#label          history painting
i DIDN'T remove this one: http://www.wikidata.org/entity/Q156890 http://www.w3.org/2000/01/rdf-schema#label Ernst Barlach
i DIDN'T remove this one: http://www.wikidata.org/entity/Q33477 http://www.w3.org/2000/01/rdf-schema#label Eug�ne Delacroix
i DIDN'T remove this one: http://www.wikidata.org/entity/Q25631150 http://www.w3.org/2000/01/rdf-schema#label              fresco
i've removed this triple: http://www.wikidata.org/entity/Q1667023 http://www.w3.org/2000/01/rdf-schema#label             offprint
i DIDN'T remove this one: https://w3id.org/artchives/MD1559580992828 http://www.w3.org/2000/01/rdf-schema#label                 N. Chaikin
i've removed this triple: http://www.wikidata.org/entity/Q131808 http://www.w3.org/2000/01/rdf-sc

i DIDN'T remove this one: https://w3id.org/artchives/MD1559574584713 http://www.w3.org/2000/01/rdf-schema#label                      G. Bosi
i DIDN'T remove this one: http://www.wikidata.org/entity/Q815410 http://www.w3.org/2000/01/rdf-schema#label        inventory
i've removed this triple: http://www.wikidata.org/entity/Q1792450 http://www.w3.org/2000/01/rdf-schema#label                     art historian
i DIDN'T remove this one: https://w3id.org/artchives/MD1559580970624 http://www.w3.org/2000/01/rdf-schema#label                 L. Cavini
i DIDN'T remove this one: https://w3id.org/artchives/MD1559657950295 http://www.w3.org/2000/01/rdf-schema#label              K. Dohrn
i DIDN'T remove this one: https://w3id.org/artchives/MD1559579577679 http://www.w3.org/2000/01/rdf-schema#label                    G.M. Clough
i've removed this triple: http://www.wikidata.org/entity/Q22132694 http://www.w3.org/2000/01/rdf-schema#label                 museum director
i've removed this triple: http://w